In [1]:
import numpy as np
from scipy import stats

In [2]:
# import networkx as nx
# import numpy as np
# from networkx.algorithms.community import louvain_communities
# def louvain(adj_matrix, num_clusters = 0):
#     graph = nx.from_numpy_array(adj_matrix)
#     comms = louvain_communities(graph)
#     labels_ = np.zeros(graph.number_of_nodes(), dtype=int)
#     for k, comm in enumerate(comms):
#         for vertex in comm: 
#             labels_[vertex] = k
#     return labels_

In [ ]:
# import numpy as np
# import networkx as nx
# import numpy.linalg as la
# from sklearn.cluster import KMeans
# import matplotlib.pyplot as plt 
# from scipy.sparse import csgraph
# def spectral_clustering(adj_matrix, num_clusters = 0): 
#     D = np.diag(np.ravel(np.sum(adj_matrix,axis=1)))
#     L = D - adj_matrix
#     l, U = la.eigh(L)
#     kmeans = KMeans(n_clusters=num_clusters).fit(U[:,1:num_clusters])
#     labels_ = kmeans.labels_
#     return labels_
# def normalized_spectral_clustering(adj_matrix, num_clusters = 0): 
#     l, U = la.eigh(csgraph.laplacian(adj_matrix, normed=True))
#     kmeans = KMeans(n_clusters=num_clusters).fit(U[:,1:num_clusters]) 
#     labels_ = kmeans.labels_
#     return labels_

In [4]:
import networkx as nx
from networkx.algorithms import tree 
import numpy as np
def get_community_labels(G):
    cc = list(nx.connected_components(G))
    labels_ = np.zeros(G.number_of_nodes(), dtype=int)
    for k, comm in enumerate(cc):
        for label in comm: 
            labels_[label] = k
    return labels_

def get_community_labels_from_less_cc(G, num_clusters):
    cc = list(nx.connected_components(G))
    labels_ = np.zeros(G.number_of_nodes(), dtype=int)
    for k, comm in enumerate(cc): 
        for label in comm:
            labels_[label] = k
    
    return labels_

def get_unique_thresholds(adj_matrix):
    tvs = adj_matrix.copy()
    tvs = tvs.reshape(tvs.shape[0] * tvs.shape[1])
    tvs = np.sort(tvs)
    tvs = np.unique(tvs)
    return tvs

def threshold_clustering(adj_matrix, num_clusters):
    adj_matrix_ = adj_matrix.copy()
    tvs = get_unique_thresholds(adj_matrix_)
    k=0
    for k in range(tvs.shape[0]):
        adj_matrix_prev = adj_matrix_.copy()
        adj_matrix_[adj_matrix_ < tvs[k]] = 0
        G = nx.from_numpy_array(adj_matrix_)
        num_components = nx.number_connected_components(G)
        if num_components < num_clusters:
            continue
        elif num_components == num_clusters:
            return get_community_labels(G)
        else:
            for i in range(adj_matrix_prev.shape[0]):
                for j in range(adj_matrix_prev.shape[0]):
                    if (adj_matrix_prev[i][j] < tvs[k] and adj_matrix_prev[i][j] != 0):
                        adj_matrix_prev[i][j] = 0
                        G = nx.from_numpy_array(adj_matrix_prev)
                        num_components = nx.number_connected_components(G)
                        if num_components == num_clusters:
                            return get_community_labels(G)
    for i in range(adj_matrix_.shape[0]):
        for j in range(adj_matrix_.shape[0]):
            if (adj_matrix_[i][j] != 0):
                adj_matrix_[i][j] = 0
                G = nx.from_numpy_array(adj_matrix_)
                num_components = nx.number_connected_components(G)
                if num_components == num_clusters:
                    return get_community_labels(G)
                
    return np.arange(adj_matrix.shape[0])


# def mst_cut_clustering(adj_matrix, num_clusters):
#     G = nx.from_numpy_array(adj_matrix)
#     mst = tree.maximum_spanning_edges(G, algorithm="kruskal")
#     edgelist = list(mst)
#     edgelist.sort(key=lambda tup: tup[2]['weight'])
#     cutted_mst = nx.from_edgelist(edgelist)
#     to_cut = edgelist[0:num_clusters - 1] 
#     cutted_mst.remove_edges_from(to_cut)
#     return get_community_labels(cutted_mst)

In [5]:
import numpy as np
from scipy import stats
# from . import parallel

# def fechner_corr(x,y):
#     x_div = x - np.mean(x)
#     y_div = y - np.mean(y)
#     return np.sum(np.sign(x_div * y_div)) / x.shape[0], 0


# Student's T random variable
# def multivariate_t_rvs(m, S, n=1, df=3):
#     '''
#     generate random variables of multivariate t distribution Parameters
#     ----------
#     m : array_like
#             mean of random variable, length determines dimension of random variable
#         S : array_like
#             square array of covariance  matrix
#     [ ]:
#     3
#         df : int or float
#             degrees of freedom
#         n : int
#             number of observations, return random array will be (n, len(m))
#         Returns
#         -------
#     rvs : ndarray, (n, len(m))
#         each row is an independent draw of a multivariate t distributed
#         random variable
#     '''
#     m = np.asarray(m)
#     d = len(m)
#     if df == np.inf:
#         x = np.ones(n)
#     else:
#         x = np.random.chisquare(df, n) / df
#     z = np.random.multivariate_normal(np.zeros(d), S, (n,))
#     return m + z/np.sqrt(x)[:,None] # same output format as random.multivariate_normal


def get_mean_cov(num_clusters = 2, cluster_size = 50, r_in = 1, r_out = 0): 
    vertex_count = num_clusters * cluster_size
    mean = np.zeros(vertex_count)
    r_ins = np.full((cluster_size, cluster_size), r_in)
    r_outs = np.full((cluster_size, cluster_size), r_out)
    cov = np.block([[np.tile(r_outs,k),r_ins,np.tile(r_outs,num_clusters-k -1)] for k in range(num_clusters)])
    np.fill_diagonal(cov,1)
    return mean, cov


def get_cor_from_cov(covariance):
    v = np.sqrt(np.diag(covariance))
    outer_v = np.outer(v, v)
    correlation = covariance / outer_v
    correlation[covariance == 0] = 0
    return correlation

def generate_samples_bag(mean, cov, bags = 10, sample_size = 50, distribution = np.random.multivariate_normal, **kwargs):
    if distribution.__name__ == 'multivariate_t_rvs' and len(kwargs):
        return np.hsplit(distribution(mean, cov, sample_size * bags, kwargs['df']).T, bags) 
    else:
        return np.hsplit(distribution(mean, cov, sample_size * bags).T, bags)
    

#this function is required since networkx does not link vertices with zero weigth
#TODO - remove this solutiuion as it slightly moves distribution if zero is in sample

In [6]:
def set_zero_weights_to_very_low(adj_matrixes, value = 1e-6):
    adj_matrixes[adj_matrixes < value] = value
    return adj_matrixes

def get_corr_estimate(sample, corr_estimator = stats.pearsonr):
    vertex_count = sample.shape[0]
    corr_estimate = np.ones((vertex_count, vertex_count))
    for i in range(vertex_count):
        for j in range(i + 1, vertex_count):
            corr, _ = corr_estimator(sample[i], sample[j])
            corr = abs(corr)
            corr_estimate[i][j] = corr
            corr_estimate[j][i] = corr
    return corr_estimate

In [7]:
# class corr_estimate_parallel(object):
#     def __init__(self, samples_bags, corr_estimator, backend = 'threading'):
#         self.samples_bags = samples_bags
#         self.corr_estimator = corr_estimator
#         self.backend = backend
#     def get_estimations(self):
#         indexes = [i for i in range(len(self.samples_bags))]
#         estimations = parallel.For(indexes, backend = self.backend, progress=False)(self.get_estimate_bag)
#         return estimations

#     def get_estimate_bag(self, index):
#         indexes = [(index, i) for i in range(len(self.samples_bags[index]))]
#         return parallel.For(indexes, backend = self.backend, progress=False)(self._estimate_sample)


#     def _estimate_sample(self, index):
#         return set_zero_weights_to_very_low(get_corr_estimate(self.samples_bags[index[0]][index[1]], self.corr_estimator)) 

# # def get_true_graph():
# #
# # # # #
# mean_covs = [get_mean_cov(num_clusters = num_clusters, cluster_size =␣ ↪cluster_size, r_in = rs[0][i], r_out = rs[1][i]) for i in range(rs.shape[1])]
#      means = [mean_cov[0] for mean_cov in mean_covs]
#      covs = [mean_cov[1] for mean_cov in mean_covs]
#      true_graph = get_cor_from_cov(cov)
#      return true_graph

In [8]:
import sklearn
import numpy as np
import networkx as nx
import numpy.linalg as la
import scipy.cluster.vq as vq
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt 
import pandas as pd
import statistics
import csv
import warnings
warnings.filterwarnings('ignore')
from tqdm.auto import tqdm
from scipy.sparse import csgraph
from scipy import stats
from sklearn.metrics.cluster import rand_score
#from sklearn.metrics import rand_score
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.metrics.cluster import mutual_info_score
import networkx.algorithms.community as nx_comm
# from .generation import get_mean_cov, get_cor_from_cov, generate_samples_bag,␣ ↪set_zero_weights_to_very_low, get_corr_estimate, multivariate_t_rvs
# from . import parallel
#from matplotlib.pyplot import figure
def get_true_labels(num_groups, num_members):
    true_labels = np.zeros(num_groups * num_members, dtype=int)
    for i in range(0, num_groups):
        for j in range(0, num_members):
            true_labels[i*num_members + j] = i
            return true_labels
        
def get_partition(labels):
    partition = []
    for i in range(labels.max() + 1):
        partition.append(np.where(labels == i)[0])
    return partition

In [9]:
def get_rs_by_relation(r_out = 0.1, relation = np.linspace( 1, 10, 10)):
    r_outs = np.full(relation.shape[0], r_out)
    r_ins = np.array([x*r_out for x in relation])
    return np.vstack((r_ins,r_outs))

def get_rs_from_fixed_rin(r_in = 0.8, count_rout = 20, epsilon = 0):
    r_ins = np.full(count_rout, r_in)
    r_outs = np.linspace( 0, r_in + epsilon, count_rout)
    return np.vstack((r_ins,r_outs))

def get_rs_from_fixed_weighted_degree(degree=16, cluster_size= 20, num_clusters=2,r_out_bound = (0,1,20)):
    #D = (N-1)D1 + (K-1)nD2
    #D1= (D - (K-1)ND2)/(N-1)
        #D2MAX > D/(KN - 1)
    r_outs = np.linspace( r_out_bound[0], r_out_bound[1], r_out_bound[2])
    r_outs = r_outs[r_outs <=degree/(num_clusters*cluster_size - 1)]
    r_ins = (degree - (num_clusters - 1)*cluster_size*r_outs)/(cluster_size - 1)
    return np.vstack((r_ins,r_outs))




In [10]:
def compute_clustering(rs, algos, num_clusters = 2, cluster_size=5, sample_vol= 10, num_repeats = 200, corr_estimator=stats.pearsonr, distribution = np.random.multivariate_normal, **kwargs):
    mean_covs = [get_mean_cov(num_clusters = num_clusters, cluster_size =cluster_size, r_in = rs[0][i], r_out = rs[1][i]) for i in range(rs.shape[1])]
    means = [mean_cov[0] for mean_cov in mean_covs]
    covs = [mean_cov[1] for mean_cov in mean_covs]
    true_graphs = [get_cor_from_cov(cov) for cov in covs] 
    
    [set_zero_weights_to_very_low(true_graph) for true_graph in true_graphs]

    samples_bags = [generate_samples_bag(means[i], covs[i], bags = num_repeats, sample_size=sample_vol, distribution = distribution, **kwargs) for i,cov in enumerate(covs)]
    print('Generating graphs started')
    estimated_graphs_bags = [[set_zero_weights_to_very_low(get_corr_estimate(sample, corr_estimator)) for sample in samples_bag] for samples_bag in tqdm(samples_bags)]
    print('Generating graphs complete')
    
    true_labels = get_true_labels(num_clusters, cluster_size)
    result = dict()
    for algo in algos:
        algo_result = []
        print(algo.__name__ + ' started')
        for idx, estimated_graphs_bag in enumerate(tqdm(estimated_graphs_bags)):
            repeat_result = []
            for estimated_graph in estimated_graphs_bag:
                repeat_result.append(algo(estimated_graph, num_clusters))
            algo_result.append(repeat_result)
        print(algo.__name__ + ' complete')
        result[algo.__name__] = algo_result
        
    return true_labels, result, estimated_graphs_bags

In [11]:
# def compute_clustering_parallel(rs, algos, num_clusters = 2, cluster_size=5, sample_vol = 10, num_repeats = 200, corr_estimator=stats.pearsonr, distribution = np.random.multivariate_normal, **kwargs):
#     mean_covs = [get_mean_cov(num_clusters = num_clusters, cluster_size = cluster_size, r_in = rs[0][i], r_out = rs[1][i]) for i in range(rs.shape[1])]
#     means = [mean_cov[0] for mean_cov in mean_covs]
#     covs = [mean_cov[1] for mean_cov in mean_covs]
#     true_graphs = [get_cor_from_cov(cov) for cov in covs] 
#     [set_zero_weights_to_very_low(true_graph) for true_graph in true_graphs]
#     samples_bags = [generate_samples_bag(means[i], covs[i], bags = num_repeats, sample_size=sample_vol, distribution = distribution, **kwargs) for i,cov in enumerate(covs)]
#     print('Generating graphs started')
#     estimated_graphs_bags = [[set_zero_weights_to_very_low(get_corr_estimate(sample, corr_estimator)) for sample in samples_bag] for samples_bag in tqdm(samples_bags)]
#     estimated_graphs_bags = parallel.For()
#     print('Generating graphs complete')
#     true_labels = get_true_labels(num_clusters, cluster_size)
#     result = dict()
#     for algo in algos:
#         algo_result = []
#         print(algo.__name__ + ' started')
#         for idx, estimated_graphs_bag in enumerate(tqdm(estimated_graphs_bags)):
#             repeat_result = []
#             for estimated_graph in estimated_graphs_bag:
#                     repeat_result.append(algo(estimated_graph, num_clusters))
#             algo_result.append(repeat_result)
        
#         print(algo.__name__ + ' complete')
#         result[algo.__name__] = algo_result

#     return true_labels, result, estimated_graphs_bags

In [12]:
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct RGB color; the keyword argument name must be a standard mpl colormap name.
    '''
    return plt.cm.get_cmap(name, n)
def nested_dict_to_dict(x):
    y = dict()
    for k1 in x:
        for k2 in x[k1]:
            y[(k1,k2)]=x[k1][k2]
    return y

In [13]:
def plot_quality_by_relation(rs, metrics, concrete_metrics=None, by_rin = True):
        #figure(figsize=(10, 12), dpi=100)
    fig, ax = plt.subplots()
    colors = ['-b', '-r', '-g', '-y', '-p', '-c', '-m']
    #if len(metrics) * len(list(metrics.values())[0]) > len(colors): reduced_metrics = nested_dict_to_dict(metrics)
    #colors = get_cmap(len(reduced_metrics))
    c=0
    if by_rin:
        r=rs[0] 
    else:
        r=rs[1] 
        if(concrete_metrics):
            for i, algo_metric in enumerate(reduced_metrics):
                if algo_metric[1] is concrete_metrics:
                    ax.plot(r, reduced_metrics[algo_metric], colors[c], label=algo_metric[1] + ' ' + algo_metric[0])
                    c+=1
                else:
                    for i, algo_metric in enumerate(reduced_metrics):
                        ax.plot(r, reduced_metrics[algo_metric], colors[c],label=algo_metric[1] + ' ' + algo_metric[0])
                        c+= 1 #ax.axis('equal')#ax.axis('p_in')
                leg = ax.legend() #plt.plot(p_ins[:len(metric)], metric) plt.ylabel('Metric value', size=10)
                if by_rin:
                    plt.xlabel('R_in', size=10)
                    plt.title('R_out=' + str(rs[1][0]), size=14)
                else:
                    plt.xlabel('R_out', size=10) #plt.title('R_out=' + str(rs[0][1]), size=14)
                    return


In [14]:
def validation(rs, true_labels, result, estimated_graphs_bags, by_rin = True):
    metrics_by_algos = dict()
    for algo in tqdm(result):
        metrics = dict()
        metrics['RI'] = [np.mean(np.array([rand_score(true_labels, labels) for labels in labels_repeated])) for labels_repeated in result[algo]]
        metrics['ARI'] = [np.mean(np.array([adjusted_rand_score(true_labels, labels) for labels in labels_repeated])) for labels_repeated in result[algo]]
        metrics['MI'] = [np.mean(np.array([mutual_info_score(true_labels, labels) for labels in labels_repeated])) for labels_repeated in result[algo]]
        metrics['AMI'] = [np.mean(np.array([adjusted_mutual_info_score(true_labels, labels) for labels in labels_repeated])) for labels_repeated in result[algo]]
        #metrics['modularity'] = [np.mean(np.array([nx_comm. ↪modularity(true_labels, get_partition(labels)) for labels in␣ ↪labels_repeated])) for labels_repeated in result[algo]]
        metrics_by_algos[algo] = metrics
    plot_quality_by_relation(rs, metrics_by_algos, 'RI', by_rin)
    plot_quality_by_relation(rs, metrics_by_algos, 'ARI', by_rin)
    plot_quality_by_relation(rs, metrics_by_algos, 'MI', by_rin)
    plot_quality_by_relation(rs, metrics_by_algos, 'AMI', by_rin)
    return metrics_by_algos

In [15]:
def metrics_to_df(metrics, rs):
    df= pd.DataFrame(nested_dict_to_dict(metrics))
    df['r_in'] = rs[0]
    df.set_index('r_in', inplace=True)
    return df


In [16]:
def analyze_mixture(r_in, r_out, algos, num_clusters = 2, cluster_size=5, sample_vol = 10, num_repeats = 200, corr_estimator=stats.pearsonr, distributions = [np.random.multivariate_normal, multivariate_t_rvs], **kwargs):
    mean, cov = get_mean_cov(num_clusters = num_clusters, cluster_size = cluster_size, r_in = r_in, r_out = r_out)
    true_graph = get_cor_from_cov(cov)
    set_zero_weights_to_very_low(true_graph)
    distr1 = np.random.multivariate_normal#distributions[0]
    distr2 = multivariate_t_rvs#distributions[1]
    print(distr1)
    samples_bag_distr1 = generate_samples_bag(mean, cov, bags = num_repeats, sample_size=sample_vol, distribution = distr1, **kwargs)
    samples_bag_distr2 = generate_samples_bag(mean, cov, bags = num_repeats, sample_size=sample_vol, distribution = distr2, **kwargs)
    print('Generating graphs started')
    estimated_graphs_distr1 = [set_zero_weights_to_very_low(get_corr_estimate(sample, corr_estimator)) for sample in samples_bag_distr1]
    estimated_graphs_distr2 = [set_zero_weights_to_very_low(get_corr_estimate(sample, corr_estimator)) for sample in samples_bag_distr2]
    print('Generating graphs complete')
    estimated_graphs_distrs = [estimated_graphs_distr1, estimated_graphs_distr2]
    true_labels = get_true_labels(num_clusters, cluster_size)
    result = dict()
    eps = np.linspace( 0, 1, 100)
    for algo in algos:
        algo_result = []
        print(algo.__name__ + ' started')
        for ep in tqdm(eps):
            repeat_result = []
            for g1,g2 in zip(estimated_graphs_distr1, estimated_graphs_distr2):
                g = g1
                if int(np.random.binomial(1, 1 - ep, 1)[0]):
                    g = g2
                    repeat_result.append(algo(g, num_clusters))
            algo_result.append(repeat_result)
        print(algo.__name__ + ' complete')
        result[algo.__name__] = algo_result
        
    return true_labels, result, estimated_graphs_distrs, eps


NameError: name 'multivariate_t_rvs' is not defined

In [19]:
def plot_quality_mixture(eps, metrics, concrete_metrics=None): #figure(figsize=(10, 12), dpi=100)
    fig, ax = plt.subplots()
    colors = ['-b', '-r', '-g', '-y', '-p', '-c', '-m']
    #if len(metrics) * len(list(metrics.values())[0]) > len(colors): reduced_metrics = nested_dict_to_dict(metrics)
    #colors = get_cmap(len(reduced_metrics))
    c=0
    if(concrete_metrics):
        for i, algo_metric in enumerate(reduced_metrics):
            if algo_metric[1] is concrete_metrics:
                ax.plot(eps, reduced_metrics[algo_metric], colors[c], label=algo_metric[1] + ' ' + algo_metric[0])
                c+=1
    else:
        for i, algo_metric in enumerate(reduced_metrics):
            ax.plot(eps, reduced_metrics[algo_metric], colors[c], label=algo_metric[1] + ' ' + algo_metric[0])
            c += 1 #ax.axis('equal')#ax.axis('p_in')
    leg = ax.legend() #plt.plot(p_ins[:len(metric)], metric)
    plt.ylabel('Metric value', size=10)
    plt.xlabel('eps', size=10)
    plt.title('Mixture uncertainty', size=14)
    return

In [20]:
def validation(true_labels, result, estimated_graphs_distrs, eps): 
    metrics_by_algos = dict()
    for algo in tqdm(result):
        metrics = dict()
        metrics['RI'] = [np.mean(np.array([rand_score(true_labels, labels) for labels in labels_repeated])) for labels_repeated in result[algo]]
        metrics['ARI'] = [np.mean(np.array([adjusted_rand_score(true_labels, labels) for labels in labels_repeated])) for labels_repeated in result[algo]]
        metrics['MI'] = [np.mean(np.array([mutual_info_score(true_labels, labels) for labels in labels_repeated])) for labels_repeated in result[algo]]
        metrics['AMI'] = [np.mean(np.array([adjusted_mutual_info_score(true_labels, labels) for labels in labels_repeated])) for labels_repeated in result[algo]]
        #metrics['modularity'] = [np.mean(np.array([nx_comm. ↪modularity(true_labels, get_partition(labels)) for labels in␣ ↪labels_repeated])) for labels_repeated in result[algo]]
        metrics_by_algos[algo] = metrics
        plot_quality_mixture(eps, metrics_by_algos, 'ARI')
        plot_quality_mixture(eps, metrics_by_algos, 'AMI')
        return metrics_by_algos, eps

# Stock clustering

In [21]:
dow30_tickers = ['AXP', 'AMGN', 'AAPL', 'BA', 'CAT',
                'CSCO', 'CVX', 'GS', 'HD', 'HON',
                'IBM', 'INTC', 'JNJ', 'KO', 'JPM',
                'MCD', 'MMM', 'MRK', 'MSFT', 'NKE',
                 'PG', 'TRV', 'UNH', 'CRM', 'VZ',
                'V', 'WBA', 'WMT', 'DIS', 'DOW']
len(dow30_tickers)

30

In [25]:
import yfinance as yf
START_DATE  ="2022-01-01"
END_DATE ="2023-01-01"
data = yf.download(' '.join(dow30_tickers), start=START_DATE, end=END_DATE,
                                 group_by='ticker')

[*********************100%***********************]  30 of 30 completed


In [26]:
data = data.apply(lambda x: np.log(x/x.shift(1)))
data = data.dropna()
data

Ticker                         CSCO                                          \
Price                          Open      High       Low     Close Adj Close   
Date                                                                          
2022-01-04 00:00:00+00:00 -0.007340 -0.008561 -0.022491 -0.030707 -0.024832   
2022-01-05 00:00:00+00:00 -0.020222 -0.018803 -0.014173 -0.015963 -0.015963   
2022-01-06 00:00:00+00:00 -0.019639 -0.008308 -0.005659  0.010561  0.010561   
2022-01-07 00:00:00+00:00  0.014396  0.006034  0.012606  0.003441  0.003441   
2022-01-10 00:00:00+00:00  0.000000  0.006483 -0.002475  0.011224  0.011224   
...                             ...       ...       ...       ...       ...   
2022-12-23 00:00:00+00:00 -0.005067  0.000000  0.006830  0.003376  0.003376   
2022-12-27 00:00:00+00:00  0.008850  0.004622  0.004457  0.001053  0.001052   
2022-12-28 00:00:00+00:00  0.000419  0.001257 -0.005096 -0.009725 -0.009725   
2022-12-29 00:00:00+00:00 -0.009057 -0.000628  0.005942  0.009094  0.009094   
2022-12-30 00:00:00+00:00  0.000212 -0.001467 -0.006581  0.002943  0.002943   

Ticker                                   MSFT                                \
Price                        Volume      Open      High       Low     Close   
Date                                                                          
2022-01-04 00:00:00+00:00  0.546561 -0.001552 -0.008318 -0.011160 -0.017296   
2022-01-05 00:00:00+00:00  0.021181 -0.027155 -0.027615 -0.031586 -0.039144   
2022-01-06 00:00:00+00:00 -0.435362 -0.039786 -0.022862 -0.014312 -0.007933   
2022-01-07 00:00:00+00:00  0.026947  0.003188 -0.006927 -0.004505  0.000510   
2022-01-10 00:00:00+00:00  0.204099 -0.014945 -0.005640 -0.017568  0.000732   
...                             ...       ...       ...       ...       ...   
2022-12-23 00:00:00+00:00 -0.883631 -0.021577 -0.012977  0.000299  0.002265   
2022-12-27 00:00:00+00:00  0.233406  0.010910  0.000251  0.008047 -0.007442   
2022-12-28 00:00:00+00:00 -0.203201 -0.007612  0.003301 -0.007064 -0.010308   
2022-12-29 00:00:00+00:00  0.146099 -0.005248  0.009136  0.006300  0.027255   
2022-12-30 00:00:00+00:00  0.146895  0.010805 -0.008135  0.004277 -0.004950   

Ticker                     ...       CVX                                \
Price                      ...       Low     Close Adj Close    Volume   
Date                       ...                                           
2022-01-04 00:00:00+00:00  ...  0.020005  0.018032  0.018032  0.426183   
2022-01-05 00:00:00+00:00  ...  0.019940  0.006485  0.006485 -0.058072   
2022-01-06 00:00:00+00:00  ... -0.003529  0.008473  0.008473 -0.150853   
2022-01-07 00:00:00+00:00  ...  0.010712  0.014258  0.014258  0.061592   
2022-01-10 00:00:00+00:00  ...  0.007374  0.000640  0.000640 -0.114980   
...                        ...       ...       ...       ...       ...   
2022-12-23 00:00:00+00:00  ...  0.031346  0.030448  0.030448  0.054788   
2022-12-27 00:00:00+00:00  ...  0.021135  0.012492  0.012492 -0.162371   
2022-12-28 00:00:00+00:00  ... -0.008321 -0.014862 -0.014863 -0.139654   
2022-12-29 00:00:00+00:00  ...  0.004084  0.007543  0.007543 -0.081629   
2022-12-30 00:00:00+00:00  ...  0.005926  0.006540  0.006540  0.189254   

Ticker                          MRK                                          \
Price                          Open      High       Low     Close Adj Close   
Date                                                                          
2022-01-04 00:00:00+00:00 -0.002353  0.004280  0.007141  0.001820  0.001820   
2022-01-05 00:00:00+00:00  0.011194  0.032343  0.017891  0.023992  0.023992   
2022-01-06 00:00:00+00:00  0.019610 -0.002886  0.008891 -0.000634 -0.000634   
2022-01-07 00:00:00+00:00  0.000888  0.011867  0.006521  0.018476  0.018476   
2022-01-10 00:00:00+00:00  0.021080  0.023562  0.015304  0.025452  0.025452   
...                             ...       ...       ...       ...       ...   
2022-12-23 00:00:00+00:00  0.003151  0

In [27]:
cov = np.cov(data.T)
cov = set_zero_weights_to_very_low(cov)

In [28]:
def check(col1, col2): 
    first1 = -1
    first2 = -1
    for i in range(len(col1)):
        if col1[i] != 0 and first1 != -1:
            first1 = i
        if col2[i] != 0 and first2 != -1:
            first2 = i

    if first1 == 0 and first2 == 0:
         return True
#
    first = max(first1, first2)
    coeff = col1[first] / col2[first]
    # -
    # elem2 = elem1 * const
    for i in range(len(col1)):
        if abs(col1[i] - col2[i] * coeff) > 1e-10:
                    return False
    return True

In [29]:
def is_invertible(matrix):
    n = len(matrix)
    for i in range(n - 1):
        for j in range(i + 1, n):
            if check(matrix[i], matrix[j]):
                return True
    return False

is_invertible(cov)

True

In [31]:
import pandas as pd
# КЛАСТЕРИЗАЦИЯ С РЕАЛЬНЫМИ ДАННЫМИ
cor = get_cor_from_cov(cov)
k=2
algos = [spectral_clustering, normalized_spectral_clustering, louvain, mst_cut_clustering ]
result = dict()
for algo in algos:
    result[algo.__name__] = algo(cor,k)
pd.DataFrame(result)

,spectral_clustering,normalized_spectral_clustering,louvain,mst_cut_clustering
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,2,0
4,0,0,2,0
...,...,...,...,...
175,0,0,0,0
176,0,0,0,0
177,0,0,2,0
178,0,0,2,0


In [35]:
import utils
from utils.generation import multivariate_t_rvs 

sample_vol = 250
distribution = np.random.multivariate_normal 
mean = np.mean(data)
distribution = multivariate_t_rvs
samples = distribution(mean, cov, sample_vol).T
esimated_cor = np.corrcoef(samples)

ModuleNotFoundError: No module named 'utils.generation'

In [37]:
k=2
algos = [spectral_clustering, normalized_spectral_clustering, louvain, mst_cut_clustering ]
result_e = dict()
for algo in algos:
    result_e[algo.__name__] = algo(esimated_cor,k)
pd.DataFrame(result)

NameError: name 'esimated_cor' is not defined

In [38]:
def stock_uncertainty(cor, num_repeats, num_clusters, algos, distribution = np.random.multivariate_normal, **kwargs):
    from utils.generation import generate_samples_bag
    from sklearn.metrics.cluster import rand_score
    #from sklearn.metrics import rand_score
    from sklearn.metrics.cluster import adjusted_rand_score
    from sklearn.metrics.cluster import adjusted_mutual_info_score
    from sklearn.metrics.cluster import mutual_info_score
    result = dict()
    sample_vol = 250
    mean = np.mean(data)
    samples_bag = generate_samples_bag(mean, cov, bags = num_repeats, sample_size=sample_vol, distribution = distribution, **kwargs)
    estimated_graphs_bag = [set_zero_weights_to_very_low(np.corrcoef(sample)) for sample in samples_bag]
   
    true_labels = dict()
    for algo in algos:
        true_labels[algo.__name__] = algo(cor, num_clusters)
        algo_result = []
        for estimated_graph in estimated_graphs_bag:
            algo_result.append(algo(estimated_graph, num_clusters))
        result[algo.__name__] = algo_result
    metrics_by_algos = dict()
    for algo in result:
        metrics = dict()
        metrics['RI'] = np.array([rand_score(true_labels[algo], labels) for labels in result[algo]]).T
        metrics['ARI'] = np.array([adjusted_rand_score(true_labels[algo], labels) for labels in result[algo]]).T
        metrics_by_algos[algo] = metrics
    return true_labels, result, estimated_graphs_bag, metrics_by_algos

In [39]:
def stock_uncertainty_different_k(cor, num_repeats, ks, algos, distribution = distribution, **kwargs):
    from utils.analyze import nested_dict_to_dict
    metrics = dict()
    for k in ks:
        true_labels, result, estimated_graphs_bag, metrics[k] = stock_uncertainty(cor, num_repeats, k, algos, distribution = distribution, **kwargs)
    for k in metrics:
        for algo in metrics[k]:
            for metric_type in metrics[k][algo]:
                metrics[k][algo][metric_type] = np.mean(metrics[k][algo][metric_type])
    metric_by_k = dict()
    for k in metrics:
        for algo in metrics[k]:
            metric_by_k[algo] = dict()
            for metric_type in metrics[k][algo]:
                metric_by_k[algo][metric_type] = []
    for k in metrics:
        for algo in metrics[k]:
            for metric_type in metrics[k][algo]:
                metric_by_k[algo][metric_type].append(metrics[k][algo][metric_type])
    df = pd.DataFrame(nested_dict_to_dict(metric_by_k), index = None)
    df['k'] = ks
    df.set_index('k', inplace=True)
    return df

NameError: name 'distribution' is not defined

In [40]:
algos = [spectral_clustering, normalized_spectral_clustering, louvain, mst_cut_clustering ]
true_labels, result, graphs, metrics = stock_uncertainty(cor, 400, 2, algos)

ModuleNotFoundError: No module named 'utils.generation'

In [41]:
from utils.analyze import nested_dict_to_dict

ModuleNotFoundError: No module named 'utils.analyze'

In [43]:
df = pd.DataFrame(nested_dict_to_dict(metrics), index = None)
df

NameError: name 'metrics' is not defined

In [45]:
metrics_by_k_normal = stock_uncertainty_different_k(cor, 400, [2,3,4,5,6,7], algos)

NameError: name 'stock_uncertainty_different_k' is not defined

In [46]:
metrics_by_k_student_df3 = stock_uncertainty_different_k(cor, 400, [2,3,4,5,6,7], algos, distribution=multivariate_t_rvs, **{'df':3})

NameError: name 'stock_uncertainty_different_k' is not defined

In [47]:
metrics_by_k_student_df3

NameError: name 'metrics_by_k_student_df3' is not defined

In [48]:
metrics_by_k_normal

NameError: name 'metrics_by_k_normal' is not defined

In [49]:
metrics_by_k_student_df2 = stock_uncertainty_different_k(cor, 400, [2,3,4,5,6,7], algos, distribution=multivariate_t_rvs, **{'df':2})
metrics_by_k_student_df2

NameError: name 'stock_uncertainty_different_k' is not defined

In [50]:
metrics_by_k_student_df2.columns

NameError: name 'metrics_by_k_student_df2' is not defined

In [51]:
metrics_by_k_student_df2 - metrics_by_k_normal

NameError: name 'metrics_by_k_student_df2' is not defined

In [52]:
#metrics_by_k_student_df2[[col for col in metrics_by_k_student_df2.columns if␣ ↪'ARI' in col[-1]]]
metrics = {'normal':metrics_by_k_normal, 'student_df3': metrics_by_k_student_df3,'student_df2':metrics_by_k_student_df2}
for name in metrics:
    metrics[name] = metrics[name][[col for col in metrics[name].columns if 'ARI' in col[-1]]]

NameError: name 'metrics_by_k_normal' is not defined

In [53]:
metrics_per_k = {}
for k in list(metrics.values())[0].index:
    metrics_per_k[k] = pd.DataFrame({name:metrics[name].loc[k] for name in metrics})

NameError: name 'metrics' is not defined

In [54]:
metrics_by_k_student_df3

NameError: name 'metrics_by_k_student_df3' is not defined

In [56]:
metrics_per_k[2]

KeyError: 2

In [57]:
metrics_per_k[3]

KeyError: 3

In [58]:
metrics_per_k[7].T.diff()

KeyError: 7

In [59]:
metrics['normal']

NameError: name 'metrics' is not defined

In [62]:
for name in metrics: 
    metrics[name].to_csv(f'{name}_distribution_overview.csv')

NameError: name 'metrics' is not defined

In [61]:
metrics_per_k

{}

In [63]:
for k in metrics_per_k:
    metrics_per_k[k].to_csv(f'detailed_view_per_{k}_clusters.csv')